In [44]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [45]:
# Load the Iris dataset
iris_df = pd.read_csv('iris.csv')

# Prepare the features
X = iris_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [46]:
# Apply k-means with K=3 (for example)
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

# Get the cluster centers
centers = kmeans.cluster_centers_

# Predict the cluster labels for the data points
labels = kmeans.predict(X)

/Users/bitaniyanassir/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [47]:
# Calculate cluster variances
variances = []
for i in range(len(centers)):
    cluster_points = X_scaled[labels == i]
    cluster_variance = cluster_points.var(axis=0)
    variances.append(cluster_variance)

In [56]:
# Estimate RBF weights
weights = []
for i in range(len(centers)):
    cluster_points = X_scaled[labels == i]
    y = cluster_points  # Target values are the points in the cluster
    X_bias = np.c_[np.ones(len(cluster_points)), cluster_points]
    weight = np.linalg.inv(X_bias.T @ X_bias) @ X_bias.T @ y
    weights.append(weight.flatten())
    

In [57]:
# Define RBF function
def rbf_function(X, centers, variances):
    distances = cdist(X, centers)

    # Reshape variances_matrix to match the dimensions of distances, except for the last dimension
    variances_matrix = np.array(variances)[:, np.newaxis, np.newaxis]  # Create a 3D matrix

    # Repeat variances_matrix along the last dimension to match the shape of distances
    variances_matrix = np.repeat(variances_matrix, distances.shape[-1], axis=-1)

    basis_values = np.exp(-distances**2 / (2 * variances_matrix**2))

    # Return a 2-dimensional array
    return basis_values[:, :, 0]  # Remove extra dimensions
 

In [58]:

def rbf_network(X, centers, variances, weights):
    rbf_features = rbf_function(X, centers, variances)
    X_reshaped = np.reshape(X, (X.shape[0], -1))  # Reshape X to have the same number of dimensions as rbf_features
    X_combined = np.hstack((X_reshaped, rbf_features))
    y_pred = np.dot(X_combined, weights.T)
    return y_pred